In [6]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re

In [22]:
import uuid

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs

In [4]:
import requests
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
from decimal import Decimal

In [97]:
#session = requests.Session()
session = boto3.Session(profile_name='personal')

right = old

In [17]:
session.profile_name

'personal'

In [15]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1',aws_access_key_id = '',
aws_secret_access_key = '')
table = dynamodb.Table('nba_bbref')

In [98]:
dynamodb = session.client('dynamodb', region_name='us-east-1')

In [24]:
%timeit [uuid.uuid4() for i in range(100)]

330 µs ± 6.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [64]:
data1 = pd.read_csv("data/bb_reference/2016to2019Data_merged.csv", encoding = 'latin-1')
data2 = pd.read_csv("data/bb_reference/2013to2015Data_merged.csv", encoding = 'latin-1')

In [65]:
print(data1.shape)
print(data2.shape)
data = data1.append(data2)
data.shape

(101014, 32)
(83567, 32)


(184581, 32)

In [66]:
data = data.drop(['Unnamed: 0_y','Unnamed: 0'], axis = 1)
data = data.rename(columns={'Unnamed: 0_x':'gamescore_rank'})

In [67]:
clnr = Cleaner()
data = clnr.transform(data)

In [68]:
data['date']=data['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [73]:
data[data['slug']=='hudsole01']

,gamescore_rank,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,...,turnovers_per_min,points_per_min,double_per_min,minutes,fg_pct,w,three_pct,two_pct,ft_pct,dt
68554,160,0,0,0,0,0,2015-03-29,0,0.0,Location.AWAY,...,0.000000,0.000000,0.000000,0.516667,0.000000,1,0.0,0.00,0.0,2015-03-29
69493,133,1,5,2,1,0,2015-04-04,3,4.1,Location.AWAY,...,0.066079,0.264317,0.198238,15.133333,0.200000,1,0.0,0.25,1.0,2015-04-04
69691,98,1,3,2,1,0,2015-04-05,3,2.7,Location.AWAY,...,0.054695,0.164084,0.109389,18.283333,0.333333,1,0.0,0.50,0.5,2015-04-05
70894,223,0,1,0,1,0,2015-04-13,0,-0.7,Location.HOME,...,0.000000,0.000000,0.000000,1.000000,0.000000,1,0.0,0.00,0.0,2015-04-13
71093,9,3,5,0,3,1,2015-04-14,1,12.5,Location.AWAY,...,0.048465,0.533118,0.048465,20.633333,0.800000,1,1.0,0.50,0.0,2015-04-14
71841,33,2,3,0,2,0,2015-04-24,1,5.0,Location.AWAY,...,0.000000,0.631579,0.000000,7.916667,0.666667,0,0.5,1.00,0.0,2015-04-24
72329,34,1,2,0,1,0,2015-05-04,0,-1.1,Location.AWAY,...,0.000000,0.000000,0.000000,3.966667,0.000000,1,0.0,0.00,0.0,2015-05-04
72406,28,2,3,0,1,1,2015-05-06,0,3.2,Location.AWAY,...,0.190174,0.475436,0.095087,10.516667,0.666667,0,1.0,0.50,0.0,2015-05-06
72450,36,2,2,0,1,0,2015-05-08,0,0.0,Location.HOME,...,0.000000,0.000000,0.000000,6.000000,0.000000,1,0.0,0.00,0.0,2015-05-08
72533,27,0,3,0,1,0,2015-05-10,0,1.9,Location.HOME,...,0.186916,0.747664,0.186916,5.350000,0.666667,1,0.0,1.00,0.0,2015-05-10


In [74]:
slug_names['hudsole01']

KeyError: 'hudsole01'

In [99]:
table = dynamodb.Table('nba_bbref')  


AttributeError: 'DynamoDB' object has no attribute 'Table'

In [84]:
dynamodb.batch_write_item()

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184581 entries, 0 to 21889
Data columns (total 54 columns):
gamescore_rank                       184581 non-null int64
assists                              184581 non-null int64
attempted_field_goals                184581 non-null int64
attempted_free_throws                184581 non-null int64
attempted_three_point_field_goals    184581 non-null int64
blocks                               184581 non-null int64
date                                 184581 non-null object
defensive_rebounds                   184581 non-null int64
game_score                           184581 non-null float64
location                             184581 non-null object
made_field_goals                     184581 non-null int64
made_free_throws                     184581 non-null int64
made_three_point_field_goals         184581 non-null int64
name                                 184581 non-null object
offensive_rebounds                   184581 non-null int64


In [70]:
players = data['slug'].unique()
names = data['player'].unique()
slug_names = {player:name for player,name in zip(players,names)}
name_slugs = {name:player for player,name in zip(players,names)}
type(players)

numpy.ndarray

In [81]:
slug_names = {}
name_slugs = {}
for row in data[['slug','player']].groupby('slug').agg('first').iterrows():
    slug_names[row[0]]=row[1]['player']
    name_slugs[row[1]['player']]=row[0]

In [22]:
players[0:20]

array(['horfoal01', 'aminual01', 'couside01', 'turneev01', 'griffbl01',
       'outlatr01', 'garcifr01', 'templga01', 'chandty01', 'bynumwi01',
       'batumni01', 'tollian01', 'martica01', 'crawfjo02', 'anthoca01',
       'koufoko01', 'brownkw01', 'maxieja01', 'artesro01', 'mcgeeja01'],
      dtype=object)

In [178]:
single = data[data['player']=='John Wall']
lag = clnr.gameplay_stats(single,player_col = 'slug',lag = 10)
lag['id']= lag['date']+lag['slug']
for ind, elem in enumerate(lag.iloc[0,:]):
    if (str(type(elem))=="<class 'numpy.float64'>"):
        lag.iloc[:,ind] = lag.iloc[:,ind].apply(lambda x : Decimal(x).quantize(Decimal('.001')))

In [89]:
name_slugs['Draymond Green']

'greendr01'

In [29]:
len(players)

1035

In [31]:
lags = [1,2,3,6,11,22,33]
for player in players[20:30]:
    now = time.time()
    for lag in lags:
        single = data[data['slug']==player]
        lag_df = clnr.gameplay_stats(single,player_col = 'slug',lag = lag)
        if len(lag_df)>0:
            lag_df['id']= lag_df['date']+lag_df['slug']
            lag_df['name']=slug_names[player]
            for ind, elem in enumerate(lag_df.iloc[0,:]):
                if (str(type(elem))=="<class 'numpy.float64'>"):
                    lag_df.iloc[:,ind] = lag_df.iloc[:,ind].apply(lambda x : Decimal(x).quantize(Decimal('.001')))
            dict_row = lag_df.iloc[0,:].to_dict()
            key  = dict_row.pop('id')
            ean = {"#"+key : key for key in dict_row.keys()}
            ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
            with table.batch_writer() as batch:
                for row in lag_df.iterrows():
                    dict_row = row[1].to_dict()
                    key  = dict_row.pop('id')
                    eav = {":"+key : dict_row[key] for key in dict_row.keys()}
                    table.update_item(Key={'id':key},
                             ExpressionAttributeNames = ean,
                             ExpressionAttributeValues = eav,
                             UpdateExpression = ue)
        print("done with {} lag ({})".format(lag,len(lag_df)))
    print("Done with {} in {:.01f}".format(slug_names[player],(time.time()-now)/60))
            

done with 1 lag (282)
done with 2 lag (282)
done with 3 lag (282)
done with 6 lag (282)
done with 11 lag (282)
done with 22 lag (282)
done with 33 lag (282)
Done with Tayshaun Prince in 6.5
done with 1 lag (543)
done with 2 lag (543)
done with 3 lag (543)
done with 6 lag (543)
done with 11 lag (543)
done with 22 lag (543)
done with 33 lag (543)
Done with LaMarcus Aldridge in 25.3
done with 1 lag (201)
done with 2 lag (201)
done with 3 lag (201)
done with 6 lag (201)
done with 11 lag (201)
done with 22 lag (201)
done with 33 lag (201)
Done with Charlie Villanueva in 9.4
done with 1 lag (356)
done with 2 lag (356)
done with 3 lag (356)
done with 6 lag (356)
done with 11 lag (356)
done with 22 lag (356)
done with 33 lag (356)
Done with Nick Young in 16.6
done with 1 lag (172)
done with 2 lag (172)
done with 3 lag (172)
done with 6 lag (172)
done with 11 lag (172)
done with 22 lag (172)
done with 33 lag (172)
Done with Steve Novak in 8.0
done with 1 lag (567)
done with 2 lag (567)
done wit

In [162]:
with table.batch_writer() as batch:
    for row in lag.iterrows():
        batch.put_item(Item = row[1].to_dict())

In [181]:
dict_row = lag.iloc[0,:].to_dict()
key  = dict_row.pop('id')
ean = {"#"+key : key for key in dict_row.keys()}
eav = {":"+key : dict_row[key] for key in dict_row.keys()}
ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
with table.batch_writer() as batch:
    for row in lag.iterrows():
        dict_row = row[1].to_dict()
        key  = dict_row.pop('id')
        #ean = {"#"+key : key for key in dict_row.keys()}
        eav = {":"+key : dict_row[key] for key in dict_row.keys()}
        #ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])
        table.update_item(Key={'id':key},
                 ExpressionAttributeNames = ean,
                 ExpressionAttributeValues = eav,
                 UpdateExpression = ue)

In [180]:
dict_row = lag.iloc[0,:].to_dict()
key  = dict_row.pop('id')
table.update_item(Key={'id':key},
                 ExpressionAttributeNames = ean,
                 ExpressionAttributeValues = eav,
                 UpdateExpression = ue)

ClientError: An error occurred (ValidationException) when calling the UpdateItem operation: Invalid UpdateExpression: An expression attribute value used in expression is not defined; attribute value: :w_06

In [172]:
ean = {"#"+key : key for key in dict_row.keys()}
eav = {":"+key : dict_row[key] for key in dict_row.keys()}
ue = "SET " + ', '.join(["#"+key +" = :"+key for key in dict_row.keys()])

In [175]:
key

'2016-01-01walljo01'

In [34]:
#lag['minutes_06'].apply(lambda x: Decimal(x))
pe = "slug, #dt, minutes_06"
fe = Key('date').lt('2016-11-07')
ea = {"#dt":"date"}
table.scan(ProjectionExpression = pe,ExpressionAttributeNames =ea, FilterExpression = fe)

{'Items': [{'slug': 'grantje01',
   'date': '2016-02-26',
   'minutes_06': Decimal('26.286')},
  {'slug': 'vucevni01', 'date': '2013-11-16', 'minutes_06': Decimal('34.789')},
  {'slug': 'pattepa01', 'date': '2015-10-30', 'minutes_06': Decimal('24.989')},
  {'slug': 'chandty01', 'date': '2015-12-23', 'minutes_06': Decimal('18.258')},
  {'slug': 'bynumwi01', 'date': '2013-11-25', 'minutes_06': Decimal('14.414')},
  {'slug': 'hansbty01', 'date': '2016-03-11', 'minutes_06': Decimal('4.681')},
  {'slug': 'villach01', 'date': '2016-02-21', 'minutes_06': Decimal('10.478')},
  {'slug': 'koufoko01', 'date': '2015-02-27', 'minutes_06': Decimal('18.919')},
  {'slug': 'outlatr01', 'date': '2014-04-02', 'minutes_06': Decimal('23.658')},
  {'slug': 'dayeau01', 'date': '2013-03-27', 'minutes_06': Decimal('7.725')},
  {'slug': 'gibsota01', 'date': '2014-03-05', 'minutes_06': Decimal('27.603')},
  {'slug': 'oubreke01', 'date': '2015-11-28', 'minutes_06': Decimal('9.95')},
  {'slug': 'carrode01', 'date'

In [90]:
#kce = Key('slug').eq('walljo01') & Key('date').lt('2016-01-20')
kce = Key('slug').eq("greendr01")
resp = table.query(IndexName = 'slug-date-index', KeyConditionExpression = kce,
            ExpressionAttributeNames =ea,
            ProjectionExpression = pe,
            ScanIndexForward = False,

    )['Items']
len(resp)

390

In [ ]:
player_data = pd.DataFrame(resp)
p

In [82]:
empties = []
for ind, slug in enumerate(players):
    kce = Key('slug').eq(slug)
    resp = table.query(IndexName = 'slug-date-index', KeyConditionExpression = kce,
            ExpressionAttributeNames =ea,
            ProjectionExpression = pe,
            ScanIndexForward = False,
            Limit = 1
    )['Items']
    if len(resp)==0:
        empties.append((ind, slug_names[slug]))
    else:
        print("{:02} {}: {}".format(ind, slug_names[slug],resp[0]['date']))

00 John Wall: 2018-12-26
12 Carmelo Anthony: 2018-11-08
24 T.J. McConnell: 2019-05-09
25 Patrick Patterson: 2019-04-07
26 Kemba Walker: 2019-04-10
27 DeMarre Carroll: 2019-04-23
28 Robert Covington: 2018-12-31
29 Chris Bosh: 2016-02-09
30 Jordan Clarkson: 2019-04-09
31 Jonas Valan?i?nas: 2019-03-31
32 Ramon Sessions: 2018-04-10
33 Larry Nance: 2019-04-09
34 Nicolas Batum: 2019-04-10
35 Kris Humphries: 2017-04-22
36 Doug McDermott: 2019-04-19
37 Garrett Temple: 2019-04-26
38 Taj Gibson: 2019-03-19
39 Wesley Matthews: 2019-04-21
40 Nikola Vu?evi?: 2019-04-23
41 Jerami Grant: 2019-04-23
42 Jahlil Okafor: 2019-04-09
43 Brandon Bass: 2017-04-25
44 Bismack Biyombo: 2019-04-10
45 Kelly Oubre: 2019-03-16
46 Cory Joseph: 2019-04-21
47 Aaron Gordon: 2019-04-23
48 Deron Williams: 2017-06-12
49 Andrew Nicholson: 2017-04-12
50 Brian Roberts: 2017-04-11
51 Jason Smith: 2019-03-15
52 Nick Young: 2018-12-22
53 Luol Deng: 2019-02-27
54 Raymond Felton: 2019-04-23
55 JaVale McGee: 2019-04-09
56 Anthony B

In [114]:
games = pd.read_csv('data/bb_reference/GameClassifier_new1.csv',encoding = 'latin-1').iloc[:,1:]
games.head()

,p1,p2,p3,team,date,venue
0,teaguje01,horfoal01,horfoal01,Team.ATLANTA_HAWKS,1/1/2013,Location.AWAY
1,bryanko01,howardw01,bryanko01,Team.LOS_ANGELES_LAKERS,1/1/2013,Location.HOME
2,holidjr01,hawessp01,holidjr01,Team.PHILADELPHIA_76ERS,1/1/2013,Location.AWAY
3,paulch01,jordade01,paulch01,Team.LOS_ANGELES_CLIPPERS,1/1/2013,Location.AWAY
4,vasqugr01,davisan02,vasqugr01,Team.NEW_ORLEANS_HORNETS,1/1/2013,Location.HOME


In [96]:
games['dt']=pd.to_datetime(games['date'])
games['date']=games['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))
#games['p1']= games['p1'].apply(lambda x:name_slugs[x])
games['p2']= games['p2'].apply(lambda x:name_slugs[x])
games['p3']= games['p3'].apply(lambda x:name_slugs[x])
games.head()

,p1,p2,p3,team,date,venue,dt
0,walljo01,gortama01,porteot01,Team.WASHINGTON_WIZARDS,2016-01-01,Location.HOME,2016-01-01
1,butleji01,gasolpa01,mirotni01,Team.CHICAGO_BULLS,2016-01-01,Location.HOME,2016-01-01
2,wadedw01,whiteha01,denglu01,Team.MIAMI_HEAT,2016-01-01,Location.HOME,2016-01-01
3,smithis01,noelne01,covinro01,Team.PHILADELPHIA_76ERS,2016-01-01,Location.AWAY,2016-01-01
4,matthwe02,pachuza01,nowitdi01,Team.DALLAS_MAVERICKS,2016-01-01,Location.AWAY,2016-01-01


In [111]:
games.head()

,p1,p2,p3,team,date,venue,dt
0,walljo01,gortama01,porteot01,Team.WASHINGTON_WIZARDS,2016-01-01,Location.HOME,2016-01-01
1,butleji01,gasolpa01,mirotni01,Team.CHICAGO_BULLS,2016-01-01,Location.HOME,2016-01-01
2,wadedw01,whiteha01,denglu01,Team.MIAMI_HEAT,2016-01-01,Location.HOME,2016-01-01
3,smithis01,noelne01,covinro01,Team.PHILADELPHIA_76ERS,2016-01-01,Location.AWAY,2016-01-01
4,matthwe02,pachuza01,nowitdi01,Team.DALLAS_MAVERICKS,2016-01-01,Location.AWAY,2016-01-01


In [112]:

both =data.merge(games[['p1','p2','p3','team','date']],how = 'inner', 
                 left_on = ['team','date'],right_on = ['team','date'])
both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101014 entries, 0 to 101013
Data columns (total 57 columns):
gamescore_rank                       101014 non-null int64
assists                              101014 non-null int64
attempted_field_goals                101014 non-null int64
attempted_free_throws                101014 non-null int64
attempted_three_point_field_goals    101014 non-null int64
blocks                               101014 non-null int64
date                                 101014 non-null object
defensive_rebounds                   101014 non-null int64
game_score                           101014 non-null float64
location                             101014 non-null object
made_field_goals                     101014 non-null int64
made_free_throws                     101014 non-null int64
made_three_point_field_goals         101014 non-null int64
name                                 101014 non-null object
offensive_rebounds                   101014 non-null int64

In [98]:
#data.info()
time.time()

1568994034.703226

In [ ]:
alpha = 0.3
n = 561
f2 = 'dk_score'
combos = [(('dk_per_k','median'),('dk_per_k','mean')),
          (('dk_per_k','median'),('dk_score','median')),
          (('dk_per_k','median'),('salary','mean')),
          (('salary','mean'),('dk_per_k','mean')),
          #('salary','dk_per_k'),
          #('minutes','dk_per_min'),
          #('salary','dk_score'),
          #('salary','minutes')
          #('salary','dk_per_min'),
          #('dk_score','dk_per_min')
         ]
fig, ax = plt.subplots(len(combos),1,figsize=(24,36))
buckets = [get_n_scatter(summary,elem[0],elem[1],ax = ax[ind],alpha= alpha,n = n, s = 44) for ind,elem in enumerate(combos)]
#buckets = get_n_scatter(data,f1,f2,ax = ax[0], alpha , n )
plt.show()

In [91]:
data.sort_values(by = 'date',inplace=True)

In [99]:
df1 = pd.read_csv('data/bb_reference/2013to2015Data_merged.csv', encoding="latin-1")
df2 = pd.read_csv('data/bb_reference/2016to2019Data_merged.csv', encoding="latin-1")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83567 entries, 0 to 83566
Data columns (total 32 columns):
Unnamed: 0                           83567 non-null int64
Unnamed: 0_x                         83567 non-null int64
assists                              83567 non-null int64
attempted_field_goals                83567 non-null int64
attempted_free_throws                83567 non-null int64
attempted_three_point_field_goals    83567 non-null int64
blocks                               83567 non-null int64
date                                 83567 non-null object
defensive_rebounds                   83567 non-null int64
game_score                           83567 non-null float64
location                             83567 non-null object
made_field_goals                     83567 non-null int64
made_free_throws                     83567 non-null int64
made_three_point_field_goals         83567 non-null int64
name                                 83567 non-null object
offensive_reboun

In [101]:
df2.head()

,Unnamed: 0,Unnamed: 0_x,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,...,team,turnovers,Unnamed: 0_y,player,birthday,year_min,year_max,pos,height,weight
0,0,0,13,19,4,3,1,1/1/2016,1,22.1,...,Team.WASHINGTON_WIZARDS,5,28,John Wall,6-Sep-90,2011,2019,G,4-Jun,210.0
1,1,54,5,21,2,6,0,1/3/2016,4,5.8,...,Team.WASHINGTON_WIZARDS,0,28,John Wall,6-Sep-90,2011,2019,G,4-Jun,210.0
2,2,57,12,19,2,4,0,1/6/2016,3,11.6,...,Team.WASHINGTON_WIZARDS,7,28,John Wall,6-Sep-90,2011,2019,G,4-Jun,210.0
3,3,71,4,20,3,5,3,1/8/2016,2,8.3,...,Team.WASHINGTON_WIZARDS,7,28,John Wall,6-Sep-90,2011,2019,G,4-Jun,210.0
4,4,7,10,17,1,6,0,1/9/2016,5,20.8,...,Team.WASHINGTON_WIZARDS,7,28,John Wall,6-Sep-90,2011,2019,G,4-Jun,210.0


In [103]:
df1= df1.append(df2).copy(deep=True)

In [105]:
def get_season(inp_date):
    if int(inp_date.split('/')[0]) > 8:
        return inp_date.split('/')[2] + '-' + str(int(inp_date.split('/')[2]) + 1)
    return str(int(inp_date.split('/')[2]) - 1) + '-' + inp_date.split('/')[2]

In [106]:
df1['min_played'] = df1['seconds_played']/60
df1['season'] = df1['date'].apply(get_season)

team_list = df1['team'].unique()
season_list = df1['season'].unique()
top_3_list = []
team = team_list[0]

In [107]:
team

'Team.ATLANTA_HAWKS'

In [108]:
for team in team_list:
    for season in season_list:
        team_df = df1[df1['team'] == team]
        team_df = team_df[team_df['season'] == season]
        team_df_G = team_df[team_df['pos'].str.contains('G')]
        team_df_C = team_df[team_df['pos'].str.contains('C')]
        team_df_grouped_G = team_df_G.groupby(['slug', 'pos']).sum()
        team_df_grouped_C = team_df_C.groupby(['slug', 'pos']).sum()
        team_df_sorted_G = team_df_grouped_G.sort_values(['game_score'], ascending=False)
        team_df_sorted_C = team_df_grouped_C.sort_values(['game_score'], ascending=False)
        top_G = list(team_df_sorted_G.head(1).index.values)[0][0]
        top_C = list(team_df_sorted_C.head(1).index.values)[0][0]
        team_df = team_df[team_df['player'] != top_G]
        team_df = team_df[team_df['player'] != top_C]
        team_df_grouped = team_df.groupby(['slug', 'pos']).sum()
        team_df_sorted = team_df_grouped.sort_values(['game_score'], ascending=False)
        other_top = list(team_df_sorted.head(1).index.values)[0][0]
        top_3_list.append([team, season, top_G, top_C, other_top])

IndexError: list index out of range

In [109]:
team

'Team.NEW_ORLEANS_HORNETS'

In [110]:
season

'2013-2014'